In [8]:
import simpy
import random
import numpy as np

<h2 style="text-align: center; font-size: 30px; text-align: center; margin-bottom: 40px;">Trabajo Práctico 2</h2>

<div style="border: 1px solid #d2d2d2; border-top: none; border-radius: 8px; margin-bottom: 40px;">
    <h2 style="text-align: center; font-weight: bold; font-size: 20px; padding: 10px 20px; background-color: #38a2ad; color: #fff; border-radius: 4px; margin-bottom: 20px; margin-top: 0; border-bottom-left-radius: 0;  border-bottom-right-radius: 0;">Alumnos</h2>
    <div style="width: 450px; margin: 0 auto 30px;">
        <div style="overflow: hidden; border-bottom: 1px solid #d2d2d2;">
            <div style="float: left; width: 300px; padding-left: 8px;">
                <p style="font-family: 'Open Sans', sans-serif; font-size: 18px; font-weight: bold; color: #363636; line-height: 40px;">Nombre y Apellido</p>
            </div>
            <div style="float: left; width: 150px;">
                <p style="font-family: 'Open Sans', sans-serif; font-size: 18px; font-weight: bold; color: #363636; line-height: 40px;">Padrón</p>
            </div>
        </div>
        <div style="overflow: hidden; border-bottom: 1px solid #d2d2d2;">
            <div style="float: left; width: 300px; padding-left: 8px;">
                <p style="font-family: 'Open Sans', sans-serif; font-size: 16px; color: #363636; line-height: 40px;">Maria Florencia Prado</p>
            </div>
            <div style="float: left; width: 150px;">
                <p style="font-family: 'Open Sans', sans-serif; font-size: 16px; color: #363636; line-height: 40px;">96626</p>
            </div>
        </div>
         <div style="overflow: hidden; border-bottom: 1px solid #d2d2d2; background-color: #f4f4f4;">
            <div style="float: left; width: 300px; padding-left: 8px;">
                <p style="font-family: 'Open Sans', sans-serif; font-size: 16px; color: #363636; line-height: 40px;">Risaro Lucas</p>
            </div>
            <div style="float: left; width: 150px;">
                <p style="font-family: 'Open Sans', sans-serif; font-size: 16px; color: #363636; line-height: 40px;">94335</p>
            </div>
        </div>
         <div style="overflow: hidden; border-bottom: 1px solid #d2d2d2;">
            <div style="float: left; width: 300px; padding-left: 8px;">
                <p style="font-family: 'Open Sans', sans-serif; font-size: 16px; color: #363636; line-height: 40px;">Scoccimarro Maximiliano</p>
            </div>
            <div style="float: left; width: 150px;">
                <p style="font-family: 'Open Sans', sans-serif; font-size: 16px; color: #363636; line-height: 40px;">93784</p>
            </div>
        </div>
    </div>
</div>

<h2 style="text-align: center; font-weight: bold; font-size: 20px; padding: 10px 20px; background-color: #db2e2e; color: #fff; border-radius: 4px; margin-bottom: 30px;">Grupo 4</h2>

# Procesos de Poisson

### Ejercicio 1

Utilizando Matlab, Octave o Python simule el siguiente sistema.

Se está diseñando un web service, el cual cada vez que es invocado consulta a una base de datos.

Se estima que el tiempo que transcurre entre cada llamada al servicio se puede modelar según una distribución exponencial con media $\mu = 4$ *segundos*

Se debe decidir la arquitectura de base de datos a utilizar entre las dos siguientes:

1) Utilizar 2 bases de datos distribuidas.

Con probabilidad $p = 0.6$ las solicitudes son atendidas por la base 1 y con probabilidad $q=1-p$ son atendidos por la base de datos 2.

El tiempo que demora cada base de datos en atender una solicitud sigue una distribución exponencial con medias,  $\mu_1 = 0.7$ *segundos* y $\mu_1 = 1$ *segundos* respectivamente.

2) Utilizar 1 base de datos central. 

En este caso la demora en resolver una solicitud sigue una distribución exponencial con $\mu = 0.8$ *segundos*

Simular para cada opción 100000 solicitudes procesadas, determinando:
  * El tiempo medio de espera entre que la solicitud llega y puede ser procesada (suponer que ninguna conexión se cae por timeout).
  * La fracción de las solicitudes que no esperaron para ser procesadas. 
  * La opción 1 es más costosa que la segunda opción y la empresa sólo acepta realizar la inversión si el tiempo medio que demora en resolver cada solicitud (tiempo en fila + tiempo de procesamiento) es como mínimo 50% menor que la opción 2. ¿Qué solución le recomienda?

In [4]:
def serviceCall():
  return np.random.exponential(1/4)

def distributedDBProcessTime():
  if (np.random.rand() <= 0.6):
    return np.random.exponential(10/7)
  else:
    return np.random.exponential(1)

def centralDBProcessTime():
  return np.random.exponential(10/8)
  
def meanInstantProcessedServiceCalls(waitTimes):
  return waitTimes.count(0) / len(waitTimes)

def simulateWaitTimes(processFn, times):
  serviceCallTime = 0
  readyAtTime = 0
  waits = list()
  solves = list()

  for i in range(0, times):
    serviceCallTime += serviceCall()
    processTime = processFn()
    waitTime = readyAtTime - serviceCallTime

    if (waitTime > 0):
      waits.append(waitTime)
      solves.append(waitTime + processTime)
    else:
      waits.append(0)
      solves.append(processTime)

    readyAtTime = serviceCallTime + processTime

  return { "waitTimes": waits, "solveTimes": solves }

def getTimeDiffPercent(t1, t2):
  wt1 = np.mean(t1)
  wt2 = np.mean(t2)
  return wt1*100/wt2

distributedTimes = simulateWaitTimes(distributedDBProcessTime, 100000)
centralTimes = simulateWaitTimes(centralDBProcessTime, 100000)

print('Tiempo medio de espera - 2 bases de datos distribuidas', np.mean(distributedTimes["waitTimes"]))
print('Tiempo medio de espera - 1 base de datos central', np.mean(centralTimes["waitTimes"]))

print('Fracción de solicitudes que no esperaron para ser procesadas - 2 bases de datos distribuidas', meanInstantProcessedServiceCalls(distributedTimes["waitTimes"]))
print('Fracción de solicitudes que no esperaron para ser procesadas - 1 base de datos central', meanInstantProcessedServiceCalls(centralTimes["waitTimes"]))

print('Tiempo medio de resolucion - 2 bases de datos distribuidas', np.mean(distributedTimes["solveTimes"]))
print('Tiempo medio de resolucion - 1 base de datos central', np.mean(centralTimes["solveTimes"]))

td = getTimeDiffPercent(distributedTimes["solveTimes"], centralTimes["solveTimes"])
print('La opcion de 2 bases de datos distribuidas es', td, '% de la opcion de 1 base de datos central')
if (td >= 50):
  print('Por lo tanto recomiendo la opcion de 1 base de datos central')
else:
  print('Por lo tanto recomiendo la opcion de 2 bases de datos distribuidas')



Tiempo medio de espera - 2 bases de datos distribuidas 1.0461186764761594
Tiempo medio de espera - 1 base de datos central 1.0378516664871644
Fracción de solicitudes que no esperaron para ser procesadas - 2 bases de datos distribuidas 0.16801
Fracción de solicitudes que no esperaron para ser procesadas - 1 base de datos central 0.16744
Tiempo medio de resolucion - 2 bases de datos distribuidas 2.299627878599376
Tiempo medio de resolucion - 1 base de datos central 2.2830132236491436
La opcion de 2 bases de datos distribuidas es 100.72775114826868 % de la opcion de 1 base de datos central
Por lo tanto recomiendo la opcion de 1 base de datos central


# Cadenas de Markov

### Ejercicio 2

Un servidor recibe solicitudes las cuales son procesadas de una por vez en el orden de llegada (política FIFO).

Se determinó que en 10 *milisegundos* existe una probabilidad $p = \frac{1}{40}$ que llegue una nueva solicitud y una probabilidad $q = \frac{1}{30}$ que una solicitud termine de ser procesada y deje el sistema.

Se desea estudiar la cantidad de solicitudes en el servidor considerando tanto las que están en cola esperando ser procesadas como la solicitud que está siendo procesada.

1)  Determine la matriz de transición de estados explicando cómo se obtiene la misma.

2)  Utilizando Matlab, Octave o Python simule la evolución del sistema a lo largo de 1.000 segundos. Suponga que el sistema puede tener como máximo 30 estados posibles y que el servidor comienza sin estar procesando solicitudes.

3)  Realice un gráfico mostrando la cantidad de solicitudes en el servidor en cada instante de tiempo.

4)  Realice un histograma mostrando cuantas veces el sistema estuvo en cada estado.

5)  Determine el % de tiempo que el servidor se encuentra sin procesar solicitudes.

# Sistemas Dinámicos

### Ejercicio 3

Se tiene un sistema el cual depende de 3 variables y se puede suponer discreto en el tiempo. Su comportamiento está regido por las siguientes reglas:

<h4 align="center">
$2x_t = x_{t-1} + 2 y_{t-1}$
</h4>
<h4 align="center">
$y_t = y_{t-1} - \frac{x_{t-1}}{2}$
</h4>
<h4 align="center">
$z_t = z_{t-1} - (x_{t-1} + y_{t-1})$
</h4>

* Determine el o los puntos de equilibrio del sistema.
* Grafique el diagrama de fases mostrando 100 iteraciones del sistema considerando como puntos de inicio todos
los puntos que se puede formar con:
<h4 align="center">
$x_0 =  \text{-1, 0 y 1}$
</h4>
<h4 align="center">
$y_0 =  \text{-1, 0 y 1}$
</h4>
<h4 align="center">
$z_0 =  \text{-1, 0 y 1} $
</h4>

# Simpy

### Ejercicio 4

A un cajero automático (ATM) arriban clientes según una distribución exponencial con media que depende del horario según la siguiente tabla:

| Hora desde | Hora hasta  | Media (seg) 
|--|
| 10:00 | 12:00| 240
| 12:00 | 15:00| 120
| 15:00 | 19:00| 360

Los clientes se pueden dividir en tres tipos, y en función de ello es el tiempo que utilizan el cajero

| Tipo | Probabilidad  | Tiempo de proceso (min) 
|--|
| 1 | .1| 4 +/- 3
| 2 | .7| 2 +/- 1
| 3 | .2| 3 +/- 2

Determine:

* La cantidad máxima de personas en fila esperando a usar el ATM
* Tiempo máximo de espera en fila



In [9]:
simulation_time = 7*60*60
# index = 1

def getLambda(env):
    if(env.now <= 2*60*60):
        return 1/240
    elif(env.now <= 5*60*60):
        return 1/120
    elif(env.now <= 7*60*60):
        return 1/360
    
    
def duracionAtencion():
    u = np.random.uniform()
    if(u <= 0.1):
        print("Cliente tipo 1")
        return np.random.uniform(1,7) * 60
    elif(u <= 0.8):
        print("Cliente tipo 2")
        return np.random.uniform(1,3) * 60
    print("Cliente tipo 3")
    return np.random.uniform(1,5) * 60
    
def cajero(env):
  index = 1
  while True:
    arriboClientes = random.expovariate(getLambda(env))
    yield env.timeout(arriboClientes)
    env.process(atencion(env, index))
    index +=1

def atencion(env, index):
  print("Cliente ", index)
  duracion =  duracionAtencion()
  print("Comienzo de atencion en", round(env.now/
                                         60, 2), "minutos")
  print("Duracion de la atencion: ", "%.2f" %(duracion/60), "minutos")
  yield env.timeout(duracion)
  

environment = simpy.Environment()
environment.process(cajero(environment))
environment.run(until=simulation_time)




Cliente  1
Cliente tipo 1
Comienzo de atencion en 6.98 minutos
Duracion de la atencion:  2.17 minutos
Cliente  2
Cliente tipo 2
Comienzo de atencion en 9.57 minutos
Duracion de la atencion:  2.35 minutos
Cliente  3
Cliente tipo 2
Comienzo de atencion en 10.76 minutos
Duracion de la atencion:  2.16 minutos
Cliente  4
Cliente tipo 2
Comienzo de atencion en 16.12 minutos
Duracion de la atencion:  2.23 minutos
Cliente  5
Cliente tipo 3
Comienzo de atencion en 20.72 minutos
Duracion de la atencion:  1.56 minutos
Cliente  6
Cliente tipo 3
Comienzo de atencion en 24.64 minutos
Duracion de la atencion:  4.55 minutos
Cliente  7
Cliente tipo 2
Comienzo de atencion en 32.22 minutos
Duracion de la atencion:  2.50 minutos
Cliente  8
Cliente tipo 3
Comienzo de atencion en 38.71 minutos
Duracion de la atencion:  4.79 minutos
Cliente  9
Cliente tipo 2
Comienzo de atencion en 43.36 minutos
Duracion de la atencion:  1.03 minutos
Cliente  10
Cliente tipo 2
Comienzo de atencion en 45.69 minutos
Duracion d

### Ejercicio 5

Siguiendo una distribución Exponencial negativa de media 45 ms arriban solicitudes a una granja de servidores, compuesta por 5 servidores y un balanceador de carga.

El tiempo de procesamiento de cada solicitud dependerá del tipo de solicitud que se trate:

| Tipo | Probabilidad  | Tiempo de proceso (mseg) 
|--|
| A | .7| 120 +/- 60
| B | .2| 240 +/- 120
| C | .1| 500 +/- 300

Se necesita determinar la mejor política de asignación de procesos a utilizar en el balanceador entre las siguientes:
* Cada solicitud recibida se asigna al servidor que esté desocupado, y de no haber alguno libre la asignación se hace al que posee menos solicitudes en cola de espera.
* Utilizando una política Round Robin (la primer solicitud se asigna al servidor 1, la segunda al 2, etc).
(Justifique la respuesta midiendo todos los indicadores que considere necesarios)